In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-5v1m_ji4
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-5v1m_ji4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !pip install torch transformers wandb -qqq

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import (
            AutoTokenizer, AutoModelForCausalLM,
            TextDataset, DataCollatorForLanguageModeling,
            Trainer, TrainingArguments,
            get_cosine_schedule_with_warmup)
# from huggingface_hub.hf_api import HfAp
# import api.filter


import os
import numpy as np
import re
import pandas as pd
import tensorflow as tf
import torch
import pathlib
import random



import wandb
wandb.init(project="my-awesome-project")

wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
#Tokenize:
file_path = "cleaned_paddingtonbear_10300.txt"

tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')
block_size = tokenizer.model_max_length
train_dataset = TextDataset(tokenizer=tokenizer, file_path=file_path, block_size=block_size, overwrite_cache=True)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

#setup wanbi:
%env WANDB_PROJECT=tweet_analysis

wandb.run.name = file_path
wandb.run.save()

Token indices sequence length is longer than the specified maximum sequence length for this model (653573 > 1024). Running this sequence through the model will result in indexing errors
wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


env: WANDB_PROJECT=tweet_analysis


True

In [ ]:
!nvidia-smi

Tue Nov  8 04:33:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Train:
#Largely taken from: https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG-

ALLOW_NEW_LINES = False     # seems to work better <--- from source
LEARNING_RATE = 1.372e-4
EPOCHS = 4
seed = random.randint(0,2**32-1)
training_args = TrainingArguments(
    report_to="wandb",
    output_dir="./model_files",
    overwrite_output_dir=True,
    do_train=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    prediction_loss_only=True,
    logging_steps=5,
    save_steps=0,
    seed=seed,
    learning_rate = LEARNING_RATE
    )

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset)
#LR schedule stuff?
train_dataloader = trainer.get_train_dataloader()
num_train_steps = len(train_dataloader)
trainer.create_optimizer_and_scheduler(num_train_steps)
trainer.lr_scheduler = get_cosine_schedule_with_warmup(
    trainer.optimizer,
    num_warmup_steps=0,
    num_training_steps=num_train_steps)

# p_start, p_end = 0.4, 0.9
# def progressify(f):
#     "Control progress bar when calling f"
#     def inner(*args, **kwargs):
#         if trainer.state.epoch is not None:
#             # we only have one epoch, EPOCHS is built into dataset
#             progress.value = p_start + trainer.state.epoch * (p_end - p_start)
#         return f(*args, **kwargs)
#     return inner

# trainer.training_step = progressify(trainer.training_step)



/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
trainer.train()
wandb.finish()


***** Running training *****
  Num examples = 638
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 638
  Number of trainable parameters = 124439808
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
5,2.979500
10,2.106300
15,2.039400
20,1.924200
25,1.691800
30,1.837100
35,1.877500
40,1.933300
45,1.914700
50,1.890900




Training completed. Do not forget to share your model on huggingface.co/models =)




train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/loss,█▄▄▄▃▃▃▃▄▂▃▂▂▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▂▂▂▂▂▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,1.0
train/global_step,638


In [ ]:
trainer.model.config.task_specific_params['text-generation'] = {
    'do_sample': True,
    'min_length': 10,
    'max_length': 160,
    'temperature': 1.,
    'top_p': 0.95,
    'prefix': '<|endoftext|>'}

In [ ]:
#Save Model:
trainer.save_model()

Saving model checkpoint to ./model_files
Configuration saved in ./model_files/config.json
Model weights saved in ./model_files/pytorch_model.bin
tokenizer config file saved in ./model_files/tokenizer_config.json
Special tokens file saved in ./model_files/special_tokens_map.json


In [ ]:
a = trainer.state.log_history
print(a[0])

{'loss': 3.6418, 'learning_rate': 0.00013719847789481088, 'epoch': 0.0, 'step': 5}


# Predict:


In [ ]:
# prepare input
predictions = []
start = "Fuck off"



start_with_bos = '<|endoftext|>' + start
encoded_prompt = trainer.tokenizer(start_with_bos, add_special_tokens=False, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(trainer.model.device)

# prediction
output_sequences = trainer.model.generate(
    input_ids=encoded_prompt,
    max_length=160, #originally 160
    min_length=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=10
    )
generated_sequences = []

# decode prediction
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = trainer.tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    if not ALLOW_NEW_LINES:
        limit = text.find('\n')
        text = text[: limit if limit != -1 else None]
    generated_sequences.append(text.strip())

for i, g in enumerate(generated_sequences):
    predictions.append([start, g])

for pair in predictions:
  print(pair[1])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Fuck off Mr Curry, I miss you so much.
Fuck off. Good luck with your scrapbook. Glad you found it safely
Fuck off! I expect I might have trouble reaching #Hungary with my suitcase still packed.
Fuck off. It was nice to meet you again #Paddington
Fuck off. I like that. #Marmintofortoise
Fuck off, I'm glad I’ve left that diary under my hat. Just made that cake with my #elevenses with Aunt Lucy, thank you.
Fuck off. That sounds so kind. I can only hope that some of them get through #GBBO #Paddington2
Fuck off, man. You're very smart. Just look at me in the stairwell!
Fuck off. I know how that looks like.
Fuck off. I'll be back soon.


[['My wish is', 'My wish is to all the survivors of the earthquake relief who have been hurting for so long. The people of Puerto Rico are suffering. The people of Puerto Rico are suffering. Our thoughts are with them today.'], ['My wish is', 'My wish is with your ex-fiancee, who you will remain friends with and who will be a part of the White House family.'], ['My wish is', 'My wish is to have a pleasant and safe holiday holiday season. To the families and friends of those lost, loved and well, and to the survivors, we cannot fail.'], ['My wish is', 'My wish is that the government takes care of every aspect of our lives.'], ['My wish is', 'My wish is for everyone to end homelessness today by ending homelessness in New York City, and providing every shelter in the world.'], ['My wish is', 'My wish is that he was alive and well when he came home from Iraq, but my hope is he will come back as a hero and recommit himself to helping our fellow Americans win the war.'], ['My wish is', "My w

# New Section